# 勾配降下法の問題と対策

## 収束速度

純粋な勾配降下法(最急降下法・バッチ勾配降下法)は、1回のパラメーター更新に全てのデータセットを使って誤差を計算する。これは(少なくとも局所)最適解への収束を保証するが、計算コストが高く、そもそも全てのデータセットがメモリに収まりきれず実行不可能なこともある。

これに対して、データセットからランダムに1つのデータを選んでパラメーターを更新し、またランダムに1つのデータを更新する確率的勾配降下法という。これは毎回のパラメーター更新で必ずしも最適解に向かうことを保証しないが、全体としてみれば確率的に最適解の方向へ向かって収束していく。また、最適解へ向かって一直線にパラメーターを更新していかないので一見非効率そうだが、毎回の計算コストが小さいため、バッチ勾配降下法と比べて時間的に早く収束する。

バッチ勾配降下法と確率的勾配降下法の折衷的な手法が、1回のパラメーター更新に2つ以上のデータを用いるミニバッチ確率的勾配降下法で、通常勾配降下法や確率的勾配効果法というとこれを指すことが多い。通常の実装は、以下の通り。

- 最初にデータをシャッフルして順番に利用
 - 毎回データセットからランダムにデータを選ぶのではない
 - データを学習する回数に偏りが生じないようにするため
- エポック(全データを1回学習する)ごとにシャッフル
 - データの並びによる偏りが生じないようにするため

確率的勾配降下法は、最適解に収束してそこにとどまることはなく、最適解の周辺をうろつくことになる。実用上は、このある程度収束した値を最適解として扱って問題ない。

In [ ]:
import numpy as np
from sklearn.datasets import make_regression
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

n_sample = 100000
batch_size = 100
n_step = 20
alpha = .01

def get_loss(w, X, y):
    return np.power(y[:, np.newaxis] - X.dot(w.T), 2)

np.random.seed(0)

X, y = make_regression(n_samples=n_sample, n_features=2, n_informative=2, random_state=0)
w = np.zeros((1, 2))
history_w = np.zeros((n_step + 1, 2))
history_loss = np.zeros((n_step + 1))
history_w[0] = w

n_step_per_epoch = np.ceil(float(n_sample) / batch_size).astype(np.uint)

for step in range(n_step):
    i = int(step % n_step_per_epoch)
    if i == 0:
        shuffled = np.random.permutation(n_sample)
    begin = i * batch_size
    end = begin + batch_size if step < n_step_per_epoch - 1 else n_sample
    idx = shuffled[begin : end]
    mini_x, mini_y = X[idx], y[idx]
    loss = get_loss(w, mini_x, mini_y)
    w = w - alpha * loss.T.dot(mini_x) / batch_size
    history_w[step + 1] = w

history_loss = get_loss(history_w, X, y).mean(axis=0)

w1 = np.linspace(-200, 200, 30)
w2 = np.linspace(-200, 200, 30)
xx, yy = np.meshgrid(w1, w2)
W = np.c_[xx.ravel(), yy.ravel()]
zz = get_loss(W, X, y).mean(axis=0).reshape(xx.shape)

fig = plt.figure()
ax = Axes3D(fig, elev=90, azim=-90)

ax.plot_wireframe(xx, yy, zz, rstride=2, cstride=2, alpha=.3)
ax.contour(xx, yy, zz, zdir='z', offset=loss.min())

ax.plot(history_w[:, 0], history_w[:, 1], history_loss, marker='x', color='red')

ax.set_xlabel('$w_1$')
ax.set_ylabel('$w_2$')
ax.set_zlabel('loss')
ax.set_xticks(())
ax.set_yticks(())
ax.set_zticks(())

plt.show()

## 局所最適解・鞍点

複雑な問題・モデルでは、勾配はなだらかでなく凹凸だらけであることが多い。ここでバッチ勾配降下法を用いると、パラメーターの初期値によっては最も誤差の少ない大域最適解ではなく、局所最適解に陥ることがある。また、最適解ではないが勾配が0になる鞍点に止まって抜けられなくなることもある。

確率的勾配降下法は、必ずしも正しい勾配の方向に進まないので、バッチ勾配降下法よりは局所最適解や鞍点にはまる可能性が下がる。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

alpha = .3
n_step = 20

def J(w):
    return - np.cos(w[0]) + np.cos(w[1] * 2) + np.sin(w[1])

def J_grad(w):
    w_ = np.zeros_like(w)
    w_[0] = np.sin(w[0])
    w_[1] = - np.sin(w[1] * 2) + np.cos(w[1])
    return w_

def gradient_descent(w, alpha, n_step):
    ws = np.zeros(tuple([n_step + 1]) + w.shape)
    
    for step in range(n_step):
        ws[step] = w
        grad = alpha * J_grad(w)
        w = w - grad
    
    ws[-1] = w
    
    return ws

w1 = np.linspace(-3, 3, 100)
w2 = np.linspace(-3, 3, 100)

xx, yy = np.meshgrid(w1, w2)
zz = J(np.array([xx, yy]))

fig = plt.figure()
ax = Axes3D(fig, elev=30, azim=-20)

ax.plot_wireframe(xx, yy, zz, rstride=2, cstride=2, alpha=.3)
ax.contour(xx, yy, zz, zdir='z', offset=zz.min())

w_history1 = gradient_descent(np.array([-2, 0.6]), alpha, n_step)
w_history2 = gradient_descent(np.array([-2, 0.5]), alpha, n_step)
ax.plot(w_history1[:, 0], w_history1[:, 1], J(w_history1.T), marker='x', color='red', label='Local optimal solution')
ax.plot(w_history2[:, 0], w_history2[:, 1], J(w_history2.T), marker='x', color='green', label='Global optimal solution')

ax.legend()
ax.set_xlabel('$w_1$')
ax.set_ylabel('$w_2$')
ax.set_zlabel('cost')
ax.set_xticks(())
ax.set_yticks(())
ax.set_zticks(())

plt.show()

また、詳細は割愛するが局所最適解や鞍点にはまるのを避けるために深層学習では学習率減衰(Learning rate decay)やmomentumを導入する。

## 学習データの追加

学習データが追加されると、バッチ勾配降下法では全てのデータで学習しなければならない。確率的勾配降下法でも通常は同様に学習し直さなければならないが、常に大量の新しいデータが生成され、またデータの傾向も変化するような状況(ユーザーの嗜好変化など)では適切な学習率を設定して新しいデータでパラメーターを1回だけ更新していき、状況の変化に対応するモデルも構築できる。